In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Interface between pandas and model code

In [2]:
data=pd.DataFrame({'x0':[1,2,3,4,5],
                  'x1':[0.01,-0.01,0.25,-4.1,0.],
                  'y':[-1.5,0.,3.6,1.3,-2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [3]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [4]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [5]:
df2=pd.DataFrame(data.values,columns=['one','two','three'])
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [6]:
df3=data.copy()
df3['strings']=['a','b','c','d','e']
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [7]:
# If the data is not homogenous then the return dtype is object
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [8]:
model_cols=['x0','x1']
data.loc[:,model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [9]:
data['category']=pd.Categorical(['a','b','a','a','b'],categories=['a','b'])

In [10]:
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [11]:
dummies=pd.get_dummies(data.category,prefix='category')
data_with_dummies=data.drop('category',axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


# Creating model descreption with patsy

In [12]:
data=data.drop('category',axis=1)

In [13]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [14]:
import patsy
y,X=patsy.dmatrices('y~x0+x1',data)
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [15]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [16]:
#patsy design matrix instance are numpy ndarray with some matadata

In [17]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [18]:
# we can filter out intercept column by adding +1 to the model
patsy.dmatrices('y~x0+x1+0',data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [19]:
coef,resid,_,_=np.linalg.lstsq(X,y,rcond=None)

In [21]:
coef=pd.Series(coef.squeeze,index=X.design_info.column_names)
coef

Intercept    <built-in method squeeze of DesignMatrix objec...
x0           <built-in method squeeze of DesignMatrix objec...
x1           <built-in method squeeze of DesignMatrix objec...
dtype: object

### Data transformation in patsy formula

In [22]:
y,X=patsy.dmatrices('y~x0+np.log(np.abs(x1)+1)',data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [23]:
# for standardization(mean 0 and variance 1) and centered(subtracted by mean)
y,X=patsy.dmatrices('y~standardize(x0)+center(x1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [24]:
new_data=pd.DataFrame({'x0':[6,7,8,9],
                      'x1':[3.1,-0.5,0,2.3],
                      'y':[1,2,3,4]})
new_x=patsy.build_design_matrices([X.design_info], new_data)
new_x

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [25]:
y,X=patsy.dmatrices('y~I(x0+x1)',data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

### Categorical data and patsy

In [26]:
data=pd.DataFrame({'key1':['a','a','b','b','a','b','a','b'],
                 'key2':[0,1,0,1,0,1,0,0],
                  'v1':[1,2,3,4,5,6,7,8],
                  'v2':[-1,0,2.5,-0.5,4.,-1.2,0.2,-1.7]})
data

,key1,key2,v1,v2
0,a,0,1,-1.0
1,a,1,2,0.0
2,b,0,3,2.5
3,b,1,4,-0.5
4,a,0,5,4.0
5,b,1,6,-1.2
6,a,0,7,0.2
7,b,0,8,-1.7


In [27]:
y,X=patsy.dmatrices('v2~key1',data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [28]:
# If we omit the intercept the the columns of each category value will be included in the model design
y,X=patsy.dmatrices('v2~key1+0',data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [29]:
# numerical columns can be interpreted as categorical value categorical with the c function
y,X=patsy.dmatrices('v2~C(key2)',data)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [30]:
# Multiple categorical terms
data['key2']=data['key2'].map({0:'zero',1:'one'})
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [31]:
y,X=patsy.dmatrices('v2~key1+key2',data)

In [32]:
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [33]:
# we can use interaction of the form key1:key2
y,X=patsy.dmatrices('v2~key1+key2+key1:key2',data)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

# Introduction to statsmodel

In [34]:
'''
Models found in statsmodel are:
1. Linear models, generalized linear model, and robust linear models
2. Linear mixed effect models
3. Analysis  of varience(ANOVA) methods
4. Time series processes and state space models.
5. Generalized method of moments.
'''

'\nModels found in statsmodel are:\n1. Linear models, generalized linear model, and robust linear models\n2. Linear mixed effect models\n3. Analysis  of varience(ANOVA) methods\n4. Time series processes and state space models.\n5. Generalized method of moments.\n'

### Estimating Linear models

In [35]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [41]:
def dnorm(mean, varience, size=1):
    if isinstance(size, int):
        size=size,
    return mean+np.sqrt(varience)*np.random.randn(*size)

In [42]:
np.random.seed(12345)
N=1000
X=np.c_[dnorm(0,0.4,size=N),dnorm(0,0.6,size=N),dnorm(0,0.2,size=N)]
eps=dnorm(0,0.1,size=N)
beta=[0.1,0.3,0.5]

y=np.dot(X,beta)+eps

1000

In [43]:
X[5:]

array([[ 0.88126723,  0.25878372,  0.54218136],
       [ 0.0587601 ,  0.37852602, -0.72156758],
       [ 0.17819191, -0.13795429, -0.25668559],
       ...,
       [-0.73360144, -0.18780786,  0.176843  ],
       [ 0.3914677 , -2.36793407,  0.3426367 ],
       [ 0.86892463,  1.48598859,  0.89419751]])

In [44]:
y[:5]

array([-0.20729638,  0.42048162,  0.25041377, -0.91149048,  0.27934287])

In [47]:
X_model=sm.add_constant(X)
X_model[:5]

array([[ 1.        , -0.12946849, -0.76181948, -0.25885961],
       [ 1.        ,  0.30291036,  0.72110594,  0.82620308],
       [ 1.        , -0.32852189, -0.62872125,  0.20291837],
       [ 1.        , -0.35147471, -1.41763295, -0.13525986],
       [ 1.        ,  1.2432688 , -0.10745991,  0.62724287]])

In [49]:
# fit using statsmodel
model=sm.OLS(y,X)
results=model.fit()

In [51]:
results.params

array([0.07151196, 0.30511487, 0.53558485])

In [55]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.537
Model:                            OLS   Adj. R-squared (uncentered):              0.536
Method:                 Least Squares   F-statistic:                              385.4
Date:                Wed, 24 Mar 2021   Prob (F-statistic):                   3.75e-166
Time:                        14:14:15   Log-Likelihood:                         -258.30
No. Observations:                1000   AIC:                                      522.6
Df Residuals:                     997   BIC:                                      537.3
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0715      0.016      4.459      0.000       0.040       0.103
x2             0.3051      0.013     24.290      0.000       0.280       0.330
x3             0.5356      0.022     24.154      0.000       0.492       0.579
==============================================================================
Omnibus:                        0.699   Durbin-Watson:                   2.083
Prob(Omnibus):                  0.705   Jarque-Bera (JB):                0.671
Skew:                          -0.063   Prob(JB):                        0.715
Kurtosis:                       3.005   Cond. No.                         1.77
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [65]:
data=pd.DataFrame(X,columns=['col0','col1','col2'])
data.head()

,col0,col1,col2
0,-0.129468,-0.761819,-0.258860
1,0.302910,0.721106,0.826203
2,-0.328522,-0.628721,0.202918
3,-0.351475,-1.417633,-0.135260
4,1.243269,-0.107460,0.627243


In [75]:
# we can use statsmodels formula api and patsy formula strings
results=smf.ols('y~col0+col1+col2',data=data).fit()

In [69]:
results.params

Intercept    0.000332
col0         0.071514
col1         0.305108
col2         0.535578
dtype: float64

In [72]:
results.tvalues

Intercept     0.033406
col0          4.456541
col1         24.274153
col2         24.140553
dtype: float64

In [73]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.537
Model:                            OLS   Adj. R-squared:                  0.535
Method:                 Least Squares   F-statistic:                     384.9
Date:                Wed, 24 Mar 2021   Prob (F-statistic):          5.99e-166
Time:                        14:23:13   Log-Likelihood:                -258.30
No. Observations:                1000   AIC:                             524.6
Df Residuals:                     996   BIC:                             544.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0003      0.010      0.033      0.973      -0.019       0.020
col0           0.0715      0.016      4.457      0.000       0.040       0.103
col1           0.3051      0.013     24.274      0.000       0.280       0.330
col2           0.5356      0.022     24.141      0.000       0.492       0.579
==============================================================================
Omnibus:                        0.700   Durbin-Watson:                   2.083
Prob(Omnibus):                  0.705   Jarque-Bera (JB):                0.671
Skew:                          -0.063   Prob(JB):                        0.715
Kurtosis:                       3.005   Cond. No.                         2.24
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [74]:
results.predict(data[:5])

0   -0.380004
1    0.684506
2   -0.106311
3   -0.529777
4    0.392393
dtype: float64

### Estimating timeseries processes

In [79]:
init_x=4
import random
values=[init_x,init_x]
N=1000
b0=0.8
b1=-0.4
noise=dnorm(0,0.1,N)
for i in range(N):
    new_x=values[-1]*b0+values[-2]*b1+noise[i]
    values.append(new_x)
    
MAXLAGS=5
models=sm.tsa.AR(values)
results=models.fit(MAXLAGS)
results.params

array([-0.01057958,  0.76521592, -0.34717471, -0.02096357, -0.06433327,
        0.03911805])